<a href="https://colab.research.google.com/github/catundchat/vector_store/blob/main/text2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyPDF2 transformers jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# use huggingface text2vec-large-chinese to get sentence embedding

import PyPDF2
import os
import jieba
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import re

# load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("GanymedeNil/text2vec-large-chinese")
model = AutoModel.from_pretrained("GanymedeNil/text2vec-large-chinese")

# set original file directory
file_directory = "/content/drive/MyDrive/Colab Notebooks/Juxue_Tech/data/test_book"

# get all pdf files then read and split them into sentences
knowledge_vectors = []
for filename in os.listdir(file_directory):
    if filename.endswith('.pdf'):

        # read pdf files
        with open(os.path.join(file_directory, filename), 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()

            # split text into sentences with jieba package
            sentences = re.split('。|！|？|；|……|，|：|“|”|（|）|、|《|》|【|】|——|……|……|……|……|……|……|', text)

            # get sentence embedding
            for sentence in sentences:
                # tokenize sentence then do text cleaning
                words = jieba.cut(sentence)
                clean_sentence = ' '.join(words)
                inputs = tokenizer(clean_sentence, return_tensors="pt")
                with torch.no_grad():
                    embeddings = model(**inputs).last_hidden_state
                knowledge_vectors.append(embeddings.mean(dim=1).numpy())

# save knowledge vectors to pickle file then do serialization
with open('/content/drive/MyDrive/Colab Notebooks/Juxue_Tech/data/test_book/knowledge_vectors.pkl', 'wb') as file:
    pickle.dump(knowledge_vectors, file)




Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.727 seconds.
DEBUG:jieba:Loading model cost 0.727 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
